In [70]:
import numpy as np
import pandas as pd

In [71]:
df_train = pd.read_csv('C:/Users/sunny/Desktop/ML-NG/HE/train.csv')
df_test = pd.read_csv('C:/Users/sunny/Desktop/ML-NG/HE/test.csv')

In [72]:
#creating new feature using length and count 
cols_to_use = ['name','desc']
len_feats = ['name_len','desc_len']
count_feats = ['name_count','desc_count']

for i in np.arange(2):
    df_train[len_feats[i]] = df_train[cols_to_use[i]].apply(str).apply(len)
    df_train[count_feats[i]] = df_train[cols_to_use[i]].apply(str).apply(lambda x: len(x.split(' ')))
    df_test[len_feats[i]] = df_test[cols_to_use[i]].apply(str).apply(len)
    df_test[count_feats[i]] = df_test[cols_to_use[i]].apply(str).apply(lambda x: len(x.split(' ')))
    
df_train['keywords_len'] = df_train['keywords'].apply(str).apply(len)
df_train['keywords_count'] = df_train['keywords'].apply(str).apply(lambda x: len(x.split('-')))
df_test['keywords_len'] = df_test['keywords'].apply(str).apply(len)
df_test['keywords_count'] = df_test['keywords'].apply(str).apply(lambda x: len(x.split('-')))    

In [73]:
from easymoney.money import EasyPeasy
ep = EasyPeasy()
ep.currency_converter(df_train['goal'], from_currency=df_train['currency'], to_currency="USD")

C:\Users\sunny\Anaconda3\lib\site-packages\pandas\core\ops.py:716: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

In [44]:
#converting all the currency into USD
from currency_converter import CurrencyConverter
c = CurrencyConverter()

for i in range(len(df_train)):
    if df_train.iloc[i]['currency']!='USD':
        df_train.iloc[i]['goal']=c.convert(df_train.iloc[i]['goal'],df_train.iloc[i]['currency'],'USD')

df_train['goalUSD']=c.convert(np.array(df_train['goal']).all, np.array(df_train['currency']).all, 'USD')   
for i in range(len(df_test)):
    if df_test.iloc[i]['currency']!='USD':
        df_test.iloc[i]['goal']=c.convert(df_test.iloc[i]['goal'],df_test.iloc[i]['currency'],'USD')

C:\Users\sunny\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [74]:
#creating new feature Giventime(deadline-launchdate)
import time
import datetime
l =['deadline','state_changed_at', 'created_at', 'launched_at']
for x in l:
    df_train[x] = df_train[x].apply(lambda k:datetime.datetime.fromtimestamp(k).strftime('%d-%m-%Y'))
    df_test[x] = df_test[x].apply(lambda k:datetime.datetime.fromtimestamp(k).strftime('%d-%m-%Y'))
df_train.deadline=df_train.deadline.apply(lambda x:datetime.datetime.strptime(x,'%d-%m-%Y'))
df_test.deadline=df_test.deadline.apply(lambda x:datetime.datetime.strptime(x,'%d-%m-%Y'))
df_train.created_at=df_train.created_at.apply(lambda x:datetime.datetime.strptime(x,'%d-%m-%Y'))
df_test.created_at=df_test.created_at.apply(lambda x:datetime.datetime.strptime(x,'%d-%m-%Y'))
df_train.state_changed_at=df_train.state_changed_at.apply(lambda x:datetime.datetime.strptime(x,'%d-%m-%Y'))
df_test.state_changed_at=df_test.state_changed_at.apply(lambda x:datetime.datetime.strptime(x,'%d-%m-%Y'))
df_train['gvntime'] = (df_train.deadline-df_train.created_at).astype('timedelta64[D]')
df_test['gvntime'] = (df_test.deadline-df_test.created_at).astype('timedelta64[D]')
  

In [75]:
df_train.columns

Index(['project_id', 'name', 'desc', 'goal', 'keywords',
       'disable_communication', 'country', 'currency', 'deadline',
       'state_changed_at', 'created_at', 'launched_at', 'backers_count',
       'final_status', 'name_len', 'name_count', 'desc_len', 'desc_count',
       'keywords_len', 'keywords_count', 'gvntime'],
      dtype='object')

In [76]:
X_train = df_train[['name_len', 'goal','name_count','disable_communication', 'country','desc_len', 'desc_count',
       'keywords_len', 'keywords_count','gvntime']]
Y_train = df_train[['final_status']]
X_test = df_test[['name_len', 'goal','name_count','disable_communication', 'country','desc_len', 'desc_count',
       'keywords_len', 'keywords_count','gvntime']]

In [77]:
from sklearn.preprocessing import LabelEncoder
cls = ['disable_communication',
       'country']
comb= [X_train,X_test] 
combine = pd.concat(comb)
for c in cls:
    lb = LabelEncoder()
    lb.fit(combine[c])
    X_train[c] = lb.transform(X_train[c])
    X_test[c] = lb.transform(X_test[c])

C:\Users\sunny\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sunny\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [78]:
X_test

,name_len,goal,name_count,disable_communication,country,desc_len,desc_count,keywords_len,keywords_count,gvntime
0,9,7000.0,1,0,20,119,19,8,1,42.0
1,16,35000.0,2,0,20,119,20,16,2,36.0
2,43,49500.0,7,0,20,126,19,43,7,37.0
3,58,40000.0,8,0,20,108,17,49,6,47.0
4,18,20000.0,3,0,20,132,21,18,3,73.0
5,59,10000.0,8,0,20,125,21,22,4,41.0
6,39,1000.0,8,0,12,125,20,37,7,36.0
7,22,300.0,4,0,9,114,25,20,4,65.0
8,16,6000.0,2,0,20,38,7,16,2,30.0
9,56,8000.0,10,0,20,115,21,47,9,60.0


In [217]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
kf = KFold(n_splits=5, shuffle=False, random_state=None)
for train_index,test_index in kf.split(X_train):
    lr = LogisticRegression()
    X_train1, X_test1 = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train1, y_test1 = Y_train.iloc[train_index], Y_train.iloc[test_index]
    lr.fit(X_train1,y_train1)
    Y_pred = lr.predict(X_test1)
    TP=np.sum(y_test1.T.values == Y_pred)
    accuracy = TP/len(Y_pred)
    print(accuracy)
    

C:\Users\sunny\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.744612965874
0.825071672986
0.815777305096
0.892028114307
0.885780346821


In [71]:
from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train,Y_train,test_size=0.2, random_state=8)


In [219]:
from sklearn.svm import SVC
svcc = SVC()
svcc.fit(X_train2,y_train2)
svcc.score(X_test2,y_test2)

C:\Users\sunny\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.74091371497271807

In [80]:
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression()
lr.fit(X_train2,y_train2)
lr.score(X_test2,y_test2)

NameError: name 'X_train2' is not defined

In [81]:
lr1 = LogisticRegression()
lr1.fit(X_train,Y_train)
Y_pred1 = lr1.predict(X_test)
np.savetxt("C:/Users/sunny/Desktop/ML-NG/HE/LR2.csv",Y_pred1, delimiter=",")

C:\Users\sunny\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [97]:
from sklearn.preprocessing import OneHotEncoder
oh=OneHotEncoder(sparse=False)
oh.fit(Y_train)

Y_train1 = oh.transform(Y_train)
X_train1 = X_train.as_matrix()
X_test1 = X_test.as_matrix()
X_test1.shape,X_train1.shape,Y_train1.shape

((63465, 9), (108129, 9), (108129, 2))

In [98]:
from numpy.random import seed
seed(47)
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.utils import np_utils
def baselinemodel():
    from nltk.tag import sequential
    model =Sequential()
    model.add(Dense(8, input_dim=9, init='normal', activation='relu'))
    
    model.add(Dense(2, init='normal', activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimator = KerasClassifier(build_fn=baselinemodel, nb_epoch=100, batch_size=100, verbose=2)
estimator.fit(X_train1, Y_train1)
y_pred=estimator.predict(X_test1)
np.savetxt("C:/Users/sunny/Desktop/ML-NG/HE/LR1.csv",y_pred, delimiter=",")

Epoch 1/100
4s - loss: 0.6746 - acc: 0.6792
Epoch 2/100
2s - loss: 0.6498 - acc: 0.6805
Epoch 3/100
2s - loss: 0.6202 - acc: 0.6800
Epoch 4/100
2s - loss: 0.6119 - acc: 0.6802
Epoch 5/100
2s - loss: 0.6100 - acc: 0.6804
Epoch 6/100
2s - loss: 0.6090 - acc: 0.6804
Epoch 7/100
2s - loss: 0.6076 - acc: 0.6804
Epoch 8/100
2s - loss: 0.6065 - acc: 0.6804
Epoch 9/100
2s - loss: 0.6061 - acc: 0.6804
Epoch 10/100
2s - loss: 0.6057 - acc: 0.6804
Epoch 11/100
2s - loss: 0.6060 - acc: 0.6804
Epoch 12/100
2s - loss: 0.6045 - acc: 0.6804
Epoch 13/100
2s - loss: 0.6044 - acc: 0.6804
Epoch 14/100
3s - loss: 0.6053 - acc: 0.6804
Epoch 15/100
4s - loss: 0.6045 - acc: 0.6804
Epoch 16/100
4s - loss: 0.6038 - acc: 0.6804
Epoch 17/100
5s - loss: 0.6043 - acc: 0.6804
Epoch 18/100
4s - loss: 0.6041 - acc: 0.6804
Epoch 19/100
2s - loss: 0.6042 - acc: 0.6804
Epoch 20/100
2s - loss: 0.6035 - acc: 0.6804
Epoch 21/100
3s - loss: 0.6038 - acc: 0.6804
Epoch 22/100
3s - loss: 0.6044 - acc: 0.6803
Epoch 23/100
3s - l

In [53]:
X_test.head

<bound method NDFrame.head of            goal  disable_communication  country  gvntime  last_change
0        7000.0                      0       20     42.0          0.0
1       35000.0                      0       20     36.0          0.0
2       49500.0                      0       20     37.0          0.0
3       40000.0                      0       20     47.0          0.0
4       20000.0                      0       20     73.0          0.0
5       10000.0                      0       20     41.0          0.0
6        1000.0                      0       12     36.0          0.0
7         300.0                      0        9     65.0          0.0
8        6000.0                      0       20     30.0          0.0
9        8000.0                      0       20     60.0          0.0
10      25000.0                      0       20     33.0          0.0
11      12000.0                      0       20     30.0          0.0
12       5000.0                      0       20     30.0    